# Stable Diffusion 3.5 - Génération de Pointe

**Module :** 02-Images-Advanced  
**Niveau :** Intermédiaire/Avancé  
**Durée estimée :** 50 minutes  

## Introduction

**Stable Diffusion 3.5** représente la dernière génération de modèles de Stability AI, combinant une architecture MMDiT (Multimodal Diffusion Transformer) avec des encodeurs de texte avancés.

### Variantes SD 3.5

| Variante | Params | VRAM | Caractéristiques |
|----------|--------|------|------------------|
| **SD 3.5 Large** | 8B | 16GB+ | Meilleure qualité |
| **SD 3.5 Large Turbo** | 8B | 16GB+ | 4 steps, rapide |
| **SD 3.5 Medium** | 2.5B | 8GB+ | Balance qualité/vitesse |

### Architecture MMDiT

```
┌─────────────────────────────────────────────────────────┐
│            Stable Diffusion 3.5 Architecture            │
├─────────────────────────────────────────────────────────┤
│   Text Prompt → [CLIP-L] + [CLIP-G] + [T5-XXL]         │
│       ↓                                                 │
│   ┌───────────────────────────────────┐                │
│   │    MMDiT (Multimodal Diffusion    │                │
│   │    Transformer)                    │                │
│   │    - Joint attention text+image   │                │
│   │    - Flow Matching objective      │                │
│   └───────────────────────────────────┘                │
│       ↓                                                 │
│   [16-ch VAE] → Output Image (1024x1024)               │
└─────────────────────────────────────────────────────────┘
```

## Prérequis

- Module 00-GenAI-Environment complété
- GPU avec 8GB+ VRAM (Medium) ou 16GB+ (Large)
- Token HuggingFace avec acceptation licence SD 3.5

In [ ]:
# =============================================================================
# 1. CONFIGURATION ET IMPORTS
# =============================================================================

import os
import sys
import time
import gc
from io import BytesIO
from pathlib import Path
from datetime import datetime
from typing import Optional, Dict, List, Tuple, Any

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Chargement variables d'environnement
from dotenv import load_dotenv
load_dotenv("../.env")
load_dotenv("../00-GenAI-Environment/.env")

# Configuration
HF_TOKEN = os.getenv("HUGGINGFACE_TOKEN") or os.getenv("HF_TOKEN")

print("╔════════════════════════════════════════════════════╗")
print("║   Stable Diffusion 3.5 - Génération de Pointe     ║")
print("╚════════════════════════════════════════════════════╝")
print(f"\n📅 Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"\n🔑 HuggingFace: {'✅ Token configuré' if HF_TOKEN else '❌ Token manquant'}")

In [ ]:
# =============================================================================
# 2. DÉTECTION GPU ET CONFIGURATION
# =============================================================================

import torch

def get_gpu_info():
    """Détecte et retourne les informations GPU."""
    if not torch.cuda.is_available():
        return None, 0, "cpu"
    
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    return gpu_name, gpu_memory, "cuda"

def recommend_model(gpu_memory: float) -> str:
    """Recommande la variante SD3.5 selon la VRAM."""
    if gpu_memory >= 24:
        return "stabilityai/stable-diffusion-3.5-large"
    elif gpu_memory >= 16:
        return "stabilityai/stable-diffusion-3.5-large-turbo"
    elif gpu_memory >= 8:
        return "stabilityai/stable-diffusion-3.5-medium"
    return None

GPU_NAME, GPU_MEMORY, DEVICE = get_gpu_info()
RECOMMENDED_MODEL = recommend_model(GPU_MEMORY) if GPU_MEMORY else None
DTYPE = torch.float16 if GPU_MEMORY and GPU_MEMORY >= 16 else torch.bfloat16

if GPU_NAME:
    print(f"\n🎮 GPU: {GPU_NAME}")
    print(f"   VRAM: {GPU_MEMORY:.1f} GB")
    if RECOMMENDED_MODEL:
        print(f"   ✅ Recommandation: {RECOMMENDED_MODEL.split('/')[-1]}")
else:
    print("\n⚠️ Pas de GPU CUDA détecté")

In [ ]:
# =============================================================================
# 3. CLIENT SD 3.5
# =============================================================================

from diffusers import StableDiffusion3Pipeline

class SD35Client:
    """Client pour Stable Diffusion 3.5 avec optimisations mémoire."""
    
    def __init__(self, model_id: str = None):
        self.model_id = model_id or RECOMMENDED_MODEL
        self.pipeline = None
        self.is_turbo = "turbo" in (self.model_id or "").lower()
        
        if not self.model_id:
            raise ValueError("Pas assez de VRAM pour SD3.5 local.")
        
        print(f"🔧 SD35Client: {self.model_id.split('/')[-1]}")
        print(f"   Turbo: {'Oui' if self.is_turbo else 'Non'}")
    
    def load(self):
        """Charge le pipeline."""
        if self.pipeline:
            return
        
        print(f"\n📥 Chargement du modèle...")
        start = time.time()
        
        self.pipeline = StableDiffusion3Pipeline.from_pretrained(
            self.model_id,
            torch_dtype=DTYPE,
            token=HF_TOKEN,
            use_safetensors=True
        )
        self.pipeline.enable_model_cpu_offload()
        self.pipeline.enable_attention_slicing()
        
        print(f"   ✅ Chargé en {time.time()-start:.1f}s")
    
    def generate(self, prompt: str, negative_prompt: str = "",
                 width: int = 1024, height: int = 1024,
                 num_inference_steps: int = None,
                 guidance_scale: float = None,
                 seed: int = None) -> List[Image.Image]:
        """Génère des images."""
        self.load()
        
        if num_inference_steps is None:
            num_inference_steps = 4 if self.is_turbo else 28
        if guidance_scale is None:
            guidance_scale = 0.0 if self.is_turbo else 4.5
        if seed is None:
            seed = np.random.randint(0, 2**32)
        if not negative_prompt:
            negative_prompt = "blurry, low quality, distorted, deformed"
        
        generator = torch.Generator(device=DEVICE).manual_seed(seed)
        
        print(f"\n🎨 Génération (seed: {seed}, steps: {num_inference_steps})...")
        start = time.time()
        
        result = self.pipeline(
            prompt=prompt,
            negative_prompt=negative_prompt,
            width=width,
            height=height,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale,
            generator=generator
        )
        
        print(f"   ✅ Généré en {time.time()-start:.1f}s")
        return result.images
    
    def unload(self):
        """Libère la mémoire."""
        if self.pipeline:
            del self.pipeline
            self.pipeline = None
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            print("✅ Mémoire libérée")

# Instanciation
try:
    sd35 = SD35Client()
except ValueError as e:
    print(f"⚠️ {e}")
    sd35 = None

╔════════════════════════════════════════════════════╗
║   Stable Diffusion 3.5 - Génération de Pointe     ║
╚════════════════════════════════════════════════════╝

📅 Date: 2026-02-18 10:03:33

🔑 HuggingFace: ❌ Token manquant


## 4. Génération de Base

In [ ]:
# =============================================================================
# 4. GÉNÉRATION DE BASE
# =============================================================================

if sd35:
    base_prompt = """
    A majestic phoenix rising from flames, 
    intricate feather details, golden and crimson colors,
    magical sparks, dark fantasy background,
    highly detailed digital art, 8k resolution
    """.strip()
    
    print(f"📝 Prompt: {base_prompt[:60]}...")
    
    images = sd35.generate(prompt=base_prompt, seed=42)
    
    if images:
        plt.figure(figsize=(10, 10))
        plt.imshow(images[0])
        plt.title(f"SD 3.5 - {sd35.model_id.split('/')[-1]}", fontsize=14)
        plt.axis('off')
        plt.show()
else:
    print("⚠️ SD35 non initialisé")

## 5. Analyse CFG Scale

In [ ]:
# =============================================================================
# 5. ANALYSE CFG SCALE
# =============================================================================

if sd35 and not sd35.is_turbo:
    cfg_prompt = "A serene lake reflecting mountains, sunset, photorealistic"
    cfg_values = [2.0, 4.5, 7.0, 10.0]
    
    print(f"\n📊 Analyse CFG Scale")
    cfg_results = []
    
    for cfg in cfg_values:
        print(f"--- CFG = {cfg} ---")
        images = sd35.generate(cfg_prompt, width=768, height=768,
                               num_inference_steps=20, guidance_scale=cfg, seed=12345)
        if images:
            cfg_results.append((cfg, images[0]))
    
    if cfg_results:
        fig, axes = plt.subplots(1, len(cfg_results), figsize=(16, 5))
        for i, (cfg, img) in enumerate(cfg_results):
            axes[i].imshow(img)
            axes[i].set_title(f"CFG = {cfg}")
            axes[i].axis('off')
        plt.suptitle("Impact du Guidance Scale (CFG)", fontsize=14)
        plt.tight_layout()
        plt.show()
elif sd35:
    print("Mode Turbo: CFG fixé à 0")
else:
    print("⚠️ SD35 non initialisé")

## 6. Comparaison Inference Steps

In [ ]:
# =============================================================================
# 6. ANALYSE INFERENCE STEPS
# =============================================================================

if sd35:
    steps_prompt = "A cyberpunk city at night, neon lights, rain, cinematic"
    step_values = [1, 2, 4, 8] if sd35.is_turbo else [10, 20, 28, 40]
    
    print(f"\n⏱️ Analyse Steps ({'Turbo' if sd35.is_turbo else 'Standard'})")
    steps_results = []
    
    for steps in step_values:
        print(f"--- Steps = {steps} ---")
        start = time.time()
        images = sd35.generate(steps_prompt, width=768, height=768,
                               num_inference_steps=steps, seed=7777)
        elapsed = time.time() - start
        if images:
            steps_results.append((steps, elapsed, images[0]))
    
    if steps_results:
        fig, axes = plt.subplots(1, len(steps_results), figsize=(16, 5))
        for i, (steps, t, img) in enumerate(steps_results):
            axes[i].imshow(img)
            axes[i].set_title(f"Steps={steps}\n({t:.1f}s)")
            axes[i].axis('off')
        plt.suptitle("Impact des Inference Steps", fontsize=14)
        plt.tight_layout()
        plt.show()
else:
    print("⚠️ SD35 non initialisé")

## 7. Styles Artistiques

In [ ]:
# =============================================================================
# 7. STYLES ARTISTIQUES
# =============================================================================

if sd35:
    base_subject = "a ancient tree in a mystical forest"
    
    styles = {
        "Photorealistic": f"{base_subject}, photorealistic, natural lighting, 8k",
        "Oil Painting": f"{base_subject}, oil painting, impressionist, brushstrokes",
        "Anime": f"{base_subject}, anime style, Studio Ghibli, vibrant colors",
        "Watercolor": f"{base_subject}, watercolor painting, soft edges, pastel",
    }
    
    print("\n🎨 Styles Artistiques")
    style_results = []
    
    for style_name, prompt in styles.items():
        print(f"--- {style_name} ---")
        images = sd35.generate(prompt, width=768, height=768, seed=5555)
        if images:
            style_results.append((style_name, images[0]))
    
    if style_results:
        fig, axes = plt.subplots(2, 2, figsize=(12, 12))
        axes = axes.flatten()
        for i, (style, img) in enumerate(style_results):
            axes[i].imshow(img)
            axes[i].set_title(style, fontsize=12)
            axes[i].axis('off')
        plt.suptitle("Comparaison des Styles", fontsize=14)
        plt.tight_layout()
        plt.show()
else:
    print("⚠️ SD35 non initialisé")

## 8. Exercices Pratiques

### Exercice 1: Portrait Artistique
Créez un portrait avec le style "Oil Painting" et différentes valeurs CFG.

### Exercice 2: Comparaison Negative Prompts
Testez l'impact de différents negative prompts sur la qualité.

### Exercice 3: Exploration Ratios
Générez la même scène en 1:1, 16:9 et 9:16.

In [ ]:
# =============================================================================
# 8. ESPACE D'EXERCICES
# =============================================================================

# Décommentez pour tester:

# portrait_prompt = "Portrait of a wise elderly man, oil painting style, Rembrandt lighting"
# if sd35:
#     for cfg in [3.0, 5.0, 7.0]:
#         images = sd35.generate(portrait_prompt, guidance_scale=cfg, seed=42)
#         plt.imshow(images[0])
#         plt.title(f"CFG={cfg}")
#         plt.show()

print("📝 Décommentez le code pour commencer les exercices")

## 9. Nettoyage Mémoire

In [ ]:
# =============================================================================
# 9. NETTOYAGE
# =============================================================================

if sd35:
    print("🧹 Libération mémoire...")
    if torch.cuda.is_available():
        vram_before = torch.cuda.memory_allocated() / 1e9
    sd35.unload()
    if torch.cuda.is_available():
        vram_after = torch.cuda.memory_allocated() / 1e9
        print(f"   Libéré: {vram_before - vram_after:.1f} GB")

## 10. Récapitulatif

### Paramètres SD 3.5

| Paramètre | Standard | Turbo |
|-----------|----------|-------|
| `steps` | 28-50 | 4 |
| `guidance_scale` | 4.0-7.5 | 0.0 |
| `width/height` | 1024 | 1024 |

### Points Clés

1. **MMDiT** pour meilleure cohérence
2. **Triple encodeur** (CLIP-L, CLIP-G, T5)
3. **Turbo** pour génération rapide
4. **Toujours libérer** la mémoire après usage

In [ ]:
# =============================================================================
# FIN DU NOTEBOOK
# =============================================================================

print("\n" + "="*60)
print("   ✅ Notebook Stable Diffusion 3.5 Complété")
print("="*60)
print(f"\n📅 Terminé: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\n📚 Concepts couverts:")
print("   • Architecture MMDiT")
print("   • Variantes (Large, Turbo, Medium)")
print("   • CFG et Inference Steps")
print("   • Styles artistiques")
print("\n➡️  Prochain module: 03-Images-Orchestration/")